In [6]:
import sys
sys.path.append('/Users/spindicate/Documents/programming/zooscraper/')

In [7]:
import django
django.setup()

In [8]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [9]:
import time
import calendar
from datetime import date, datetime as dt, timedelta
from dateutil.relativedelta import relativedelta
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from django.db import transaction
from django.db.models import F, ExpressionWrapper, Avg
from django.db.models.functions import TruncDay

from zooscraper.globals import ChromeInstantiator, max_bulk_create
from casestudy.models import Region

In [5]:
region_objs = []
with ChromeInstantiator(headless=True) as chrome:
    url = 'https://covid2019app.live/live'
    chrome.get(url)
    
    drop = chrome.find_element_by_xpath("//div[@class='dropdown-menu dropdown-menu-right']")
    
    counts = {}
    btns = drop.find_elements_by_tag_name('a')
    links = [(btn.get_attribute('href'), btn.get_attribute('innerHTML')) for btn in btns][:-1]
    for link in links:
        print (link)
        chrome.get(link[0])
        
        header = WebDriverWait(chrome, 10).until(
            EC.presence_of_element_located((By.XPATH, "//tr[@class='dx-row dx-header-row']"))
        )
        col_names = header.find_elements_by_xpath(".//div[contains(@class, 'dx-datagrid-text-content')]")
        for col_name in col_names[1:]:
            region_objs.append(Region(name=col_name.get_attribute('innerHTML'), country=link[1]))        

Instantiating WebDriver...
Success
('https://covid2019app.live/country/China', 'China')
('https://covid2019app.live/country/Italy', 'Italy')
('https://covid2019app.live/country/USA', 'USA')
('https://covid2019app.live/country/Iran', 'Iran')
('https://covid2019app.live/country/France', 'France')
('https://covid2019app.live/country/Korea,%20Republic%20of', 'Korea, Republic of')
('https://covid2019app.live/country/Spain', 'Spain')
('https://covid2019app.live/country/Germany', 'Germany')
('https://covid2019app.live/country/Switzerland', 'Switzerland')
('https://covid2019app.live/country/Iraq', 'Iraq')
('https://covid2019app.live/country/United%20Kingdom', 'United Kingdom')
('https://covid2019app.live/country/Netherlands', 'Netherlands')
('https://covid2019app.live/country/Canada', 'Canada')
('https://covid2019app.live/country/Australia', 'Australia')
('https://covid2019app.live/country/Denmark', 'Denmark')


In [10]:
# max_bulk_create(region_objs, Region)

In [11]:
braz_regions = Region.objects.filter(country_key=31).exclude(name='Brazil')

In [12]:
braz_regions.count()

27

In [16]:
region_objs = []
with ChromeInstantiator(headless=False) as chrome:
    url = 'https://www.openstreetmap.org/'
    chrome.get(url)
    
    for region in braz_regions:
        location_input = chrome.find_elements_by_css_selector("input[name='query']")[1]
        location_input.clear()
        
#         country = 'South Korea' if 'Korea' in region.country else region.country
        location_input.send_keys(region.name + ', ' + region.country_key.name)
        
        find = chrome.find_elements_by_css_selector("input[value='Go']")[2]
        find.click()
        
        time.sleep(3)
        export = chrome.find_element_by_css_selector("a[id='export_tab']")
        export.click()
        
        lat = float(chrome.current_url.split('/')[-2])
        long = float(chrome.current_url.split('/')[-1])
        print (region.name, region.country_key.name, chrome.current_url)
        print ([region, lat, long])
        region_objs.append([region, lat, long])

Instantiating WebDriver...
Success
Rio Grande Do Sul Brazil https://www.openstreetmap.org/export#map=7/-30.535/-53.590
[<Region: Region object (554)>, -30.535, -53.59]
Espirito Santo Brazil https://www.openstreetmap.org/export#map=15/-30.1487/-51.2151
[<Region: Region object (555)>, -30.1487, -51.2151]
Sao Paulo Brazil https://www.openstreetmap.org/export#map=18/-23.55044/-46.63312
[<Region: Region object (556)>, -23.55044, -46.63312]
Rio De Janeiro Brazil https://www.openstreetmap.org/export#map=10/-22.9145/-43.4477
[<Region: Region object (557)>, -22.9145, -43.4477]
Parana Brazil https://www.openstreetmap.org/export#map=14/-7.5195/-72.8929
[<Region: Region object (558)>, -7.5195, -72.8929]
Para Brazil https://www.openstreetmap.org/export#map=6/-3.646/-52.420
[<Region: Region object (559)>, -3.646, -52.42]
Santa Catarina Brazil https://www.openstreetmap.org/export#map=7/-27.704/-50.974
[<Region: Region object (560)>, -27.704, -50.974]
Bahia Brazil https://www.openstreetmap.org/export#

In [17]:
with transaction.atomic():
    for region_obj in region_objs:
        print (region_obj)
        region_obj[0].latitude = region_obj[1]
        region_obj[0].longitude = region_obj[2]
        region_obj[0].save()
        print(region_obj[0].__dict__)

[<Region: Region object (554)>, -30.535, -53.59]
{'_state': <django.db.models.base.ModelState object at 0x111f7bcd0>, 'id': 554, 'name': 'Rio Grande Do Sul', 'name_alt': None, 'country_key_id': 31, 'country': '', 'country_alt': None, 'longitude': -53.59, 'latitude': -30.535, 'LAND_A_KM': None, 'land_density': None, 'city_dens': None, 'population': None, 'A85PLUSB': None, 'A80PLUSB': None, 'A75PLUSB': None, 'A70PLUSB': None, 'A65PLUSB': None, 'A09UNDERB': None, 'A14UNDERB': None, 'A19UNDERB': None, 'A24UNDERB': None, 'A29UNDERB': None, 'A34UNDERB': None, 'A05_14B': None, 'A05_19B': None, 'A05_24B': None, 'A05_29B': None, 'A05_34B': None, 'A15_24B': None, 'A15_29B': None, 'A15_34B': None, 'A20_29B': None, 'A20_34B': None, 'A35_54B': None, 'A40_54B': None, 'A45_54B': None, 'A35_64B': None, 'A40_64B': None, 'A45_64B': None, 'A80_84B': None, 'A75_79B': None, 'A70_74B': None, 'A65_69B': None, 'A60_64B': None, 'A55_59B': None, 'A50_54B': None, 'A45_49B': None, 'A40_44B': None, 'A35_39B': None

In [22]:
import pandas pd
pd.DataFrame(Region.objects.filter(country_key__alpha3='BRA').values('name', 'latitude'))


SyntaxError: invalid syntax (<ipython-input-22-de3db66756d4>, line 1)